In [1]:
import pandas as pd
import numpy as np
import datetime
from pandas_summary import DataFrameSummary

In [2]:
df = pd.read_feather('train_normalized_data.fth')
df_test = pd.read_feather('test_normalized_data.fth')

In [3]:
df_train = df[df.Date < datetime.datetime(2015, 7, 1)]  
df_val = df[df.Date >= datetime.datetime(2015, 7, 1)]

$$
\textrm{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} \left(\frac{\hat{y}_i - y_i}{y_i}\right)^2}
$$

In [4]:
def get_metric(df, sales_):
    return (((df['Sales'] - sales_)/df['Sales'])**2).sum()/len(df)

In [5]:
print('Train:')
print(get_metric(df_train, df_train['Sales'].mean()))
print('Val:')
get_metric(df_val, df_train['Sales'].mean())

Train:
0.4059077987085825
Val:


0.30213969475255925

In [6]:
stores_mean = {}
stores_mean_list = []
for store, g_df in df_train.groupby('Store'):
    store_m =  g_df[g_df['Sales'] > 0]['Sales'].mean()
    stores_mean[store] = store_m
    stores_mean_list.append(store_m)

In [7]:
print('Train:')
print(get_metric(df_train, df_train['Store'].apply(stores_mean.get)))
print('Val:')
print(get_metric(df_val, df_val['Store'].apply(stores_mean.get)))

Train:
0.15688835922756375
Val:
0.0943562401279468


In [8]:
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Embedding, Input, Flatten, Concatenate, Dense, BatchNormalization, Activation, LeakyReLU, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical

In [9]:
def rmspe(y_true, y_pred):
    return K.mean(K.square((y_true - y_pred)/y_true))

In [10]:
X_train = df_train['Store']
y_train = df_train['Sales']
X_train_cat = to_categorical(X_train)


In [11]:
X_train_cat.shape, X_train.max(),  X_train.min()

((814150, 1115), 1114, 0)

In [12]:
store_input = Input(shape=(X_train_cat.shape[1],), name="store_input")
out = Dense(1, name="Output")(store_input)
model = Model(store_input, out)
model.compile(optimizers.Adam(lr=0.01), loss='mse', metrics=[rmspe, 'mse'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
store_input (InputLayer)     [(None, 1115)]            0         
_________________________________________________________________
Output (Dense)               (None, 1)                 1116      
Total params: 1,116
Trainable params: 1,116
Non-trainable params: 0
_________________________________________________________________


In [13]:
weights = model.get_weights()

In [14]:
weights[0] = np.array(np.array(stores_mean_list).reshape(-1, 1))

In [15]:
model.set_weights(weights)

In [16]:
model.evaluate(X_train_cat, y_train)

25443/25443 [==============================] - 18s 704us/step - loss: 3864611.0000 - rmspe: 0.1569 - mse: 3864611.0000


[3864611.0, 0.15692844986915588, 3864611.0]

In [ ]:
model.fit(X_train_cat, y_train, epochs=5)

Epoch 1/5
25443/25443 [==============================] - 22s 850us/step - loss: 3864724.5000 - rmspe: 0.1569 - mse: 3864724.5000
Epoch 2/5
19369/25443 [=====================>........] - ETA: 5s - loss: 3865228.2500 - rmspe: 0.1669 - mse: 3865228.2500

In [63]:
store_input = Input(shape=(1,), name="store_input")
out = Dense(1, name="Output")(store_input)
model = Model(store_input, out)
model.compile(optimizers.Adam(lr=0.001), loss='mse', metrics=[rmspe, 'mse'])

In [64]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
25443/25443 [==============================] - 21s 814us/step - loss: 24653102.0000 - rmspe: 0.5403 - mse: 24653102.0000
Epoch 2/10
25443/25443 [==============================] - 21s 809us/step - loss: 21482116.0000 - rmspe: 0.5635 - mse: 21482116.0000
Epoch 3/10
25443/25443 [==============================] - 21s 808us/step - loss: 21402258.0000 - rmspe: 0.5615 - mse: 21402258.0000
Epoch 4/10
25443/25443 [==============================] - 21s 810us/step - loss: 21322946.0000 - rmspe: 0.5593 - mse: 21322946.0000
Epoch 5/10
25443/25443 [==============================] - 21s 811us/step - loss: 21243850.0000 - rmspe: 0.5572 - mse: 21243850.0000
Epoch 6/10
25443/25443 [==============================] - 21s 809us/step - loss: 21164960.0000 - rmspe: 0.5557 - mse: 21164960.0000
Epoch 7/10
25443/25443 [==============================] - 21s 813us/step - loss: 21086476.0000 - rmspe: 0.5537 - mse: 21086476.0000
Epoch 8/10
25443/25443 [==============================] - 21s 809us/step - l

In [15]:
X_train_cat = to_categorical(X_train)

In [16]:
X_train_cat.shape

(814150, 1115)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
store_input (InputLayer)     [(None, 1115)]            0         
_________________________________________________________________
Output (Dense)               (None, 1)                 1116      
Total params: 1,116
Trainable params: 1,116
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train_cat, y_train, epochs=10)